Probamos que la librería de acceso a archivos HDF5 de Python, `h5py`, funciona correctamente.

In [1]:
import h5py

In [2]:
#Creamos un archivo en modo escritura
f = h5py.File("prueba.h5", "w")

In [3]:
#lo cerramos: 
f.close()

## Tipos de datos que almacena un HDF5

### Grupos:

Podemos crear grupos en un archivo, aunque estos se pueden crear automáticamente al momento de guardar un dataset

In [4]:
# Abrimos el archuvo anterior en modo lectura y escritura:

f = h5py.File("prueba.h5", "r+")

In [5]:
f.create_group("Header")
f.create_group("Data")

<HDF5 group "/Data" (0 members)>

In [6]:
#Para consultar lo que hay en un archivo, usamos .keys()
f.keys()

<KeysViewHDF5 ['Data', 'Header']>

In [7]:
list(f.keys())

['Data', 'Header']

In [8]:
f.close()

### Datasets:

Podemos crear datasets vacíos (de cierto tamaño y tipo de dato definido), así como "llenos" directamente.

In [9]:
#Cargamos datos del ejemplo de la clase:

import numpy as np

rand1 = np.loadtxt("/home/sebastian/Documents/Magister/hcai2023/Codes/3.2/datos.txt", usecols=[1], unpack=True)

In [10]:
#Abrimos nuevamente el archivo en modo r+:

f = h5py.File("prueba.h5", "r+")

In [11]:
#Almacenamos nuestro array:
f["rand1"] = rand1

In [12]:
f.close()

In [13]:
#También puedo agregar datos dentro de los grupos:
f = h5py.File("prueba.h5", "r+")
f['Data/rand2'] = rand1+10.7

In [14]:
f.close()

In [15]:
#Incluso, los grupos nuevos se crean automáticamente al momento de almacenar un dataset:
f = h5py.File("prueba.h5", "r+")
f['DataBad/rand3'] = rand1/0.68

In [16]:
f.close()

Para leer los datos, utilizamos la misma estrategia que los objetos compuestos de numpy utilizando el objeto que contiene el archivo HDF5:

In [17]:
f = h5py.File("prueba.h5", "r")

In [18]:
#Utilizando el label y el operador de indexación, se puede acceder al contenido del archivo:
f['rand1'][:50]

array([-0.750514,  1.513391, -0.257402,  2.42373 , -0.937215,  0.607678,
       -0.802247, -0.03792 , -1.16919 , -1.32793 , -1.44829 ,  0.434063,
       -0.888893,  1.350377, -0.389023,  0.154864, -1.156754, -1.201204,
       -0.622379,  0.389723,  0.584301,  1.207718,  0.113014, -1.357977,
       -0.048609, -0.230104, -0.647716,  0.394307,  0.467628,  0.684698,
        1.343784,  0.045572,  0.888684,  1.378604,  0.309605, -0.115059,
        0.074366,  1.317366,  2.239764,  0.195176, -1.171153, -1.399296,
       -0.244733,  0.890791, -0.007609, -0.335468,  0.807286,  1.854221,
        0.395264,  0.748414])

In [19]:
#Para trabajar con los datos del HDF5, es conveniente copiar los valores a la memoria RAM:
mi_array = f['rand1'][:]

In [20]:
mi_array

array([-0.750514,  1.513391, -0.257402, ...,  0.629598,  0.037971,
        0.256897])

In [21]:
f.close()

Exploramos nuestro archivo creado:

In [23]:
f = h5py.File('prueba.h5', 'r')

In [24]:
f.keys()

<KeysViewHDF5 ['Data', 'DataBad', 'Header', 'rand1']>

In [25]:
#Si queremos ver qué es cada cosa, tenemos que preguntar a sus referencias respectivas
r = f['rand1']
type(r)

h5py._hl.dataset.Dataset

In [26]:
g = f['Data']
type(g)

h5py._hl.group.Group

In [27]:
#Recordamos que r es una referencia al dataset, por lo tanto NO está en memoria RAM.
#Para ellom debemos COPIAR los datos del disco a la RAM utilizando [:]
rand_ram = r[:]

In [28]:
rand_ram

array([-0.750514,  1.513391, -0.257402, ...,  0.629598,  0.037971,
        0.256897])

In [29]:
f.close()

### Atributos

Para usar los atributos, la librería incorpora un componente (objeto) que se llama `attrs`, y está asociado a todos los objetos HDF5, ya sea file, dataset y group.

Los atributos nos sirven para agregar **información** adicional que me permitan tener registro de qué son los datos incluidos. Estos son metadatos.

METADATOS útiles en el contexto astronómico:
   - Fecha de creación y lugar.
   - Versión del código que utilicé.
   - Modificaciones a la versión utilizada.
   - Parámetros utilizados.
   - Opciones de compilación.
   - Naturaleza de los datos (pro-tip, qué son?)
   - Unidades.
   - etc...

In [30]:
f = h5py.File('prueba.h5', 'r+')

In [31]:
type(f.attrs)

h5py._hl.attrs.AttributeManager

Esta referencia `f.attrs` nos permite manipular los atributos del archivo, es decir, asociados al Grupo raíz `/`.

In [32]:
#Podemos consultar los attrs de este grupo:
f.attrs.keys()

<KeysViewHDF5 []>

In [33]:
#Y podemos agregar nuevos:
f.attrs['Description'] = 'Este es un archivo de ejemplo'

In [34]:
f.attrs.keys()

<KeysViewHDF5 ['Description']>

In [35]:
f.attrs['Description']

'Este es un archivo de ejemplo'

In [36]:
#Otros ejemplos:
f.attrs['Fecha'] = '2023-11-14'

In [37]:
f.attrs['Númeroentero'] = 207

Los atributos pueden estar asociados a cualquier Grupo o Dataset de un HDF5:

In [38]:
#Ej. agregamos información a nuestro grupo "Header":
f['Header'].attrs.keys()

<KeysViewHDF5 []>

In [39]:
#Método 1:
f['Header'].attrs['Clase'] = 'HCAI 2023'

In [40]:
#Método 2:
g = f['Header']
g.attrs['Clima'] = 'Sunny'

In [41]:
#Método 3:
a = g.attrs
#También: a = f['Header'].attrs
a['Hora'] = '12:44'

In [42]:
#Consultamos los atributos creados
f['Header'].attrs.keys()

<KeysViewHDF5 ['Clase', 'Clima', 'Hora']>

In [43]:
#También, podemos asociar atributos a dataset:
f['rand1'].attrs['Description'] = 'Array de números aleatorios que siguen una distribución normal con sigma en 0'

In [44]:
f.close()